# Task 1

## Create table for SA2 Regions

In [1]:
from sqlalchemy import create_engine, text
import psycopg2 # For PostgreSQL database connections
import psycopg2.extras
import json # For handling JSON data
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt # For plotting data

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [2]:
sa2=gpd.read_file("SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
sa2

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [3]:
def create_wkt_element(geom, srid=4283):
    if geom is not None:
        if geom.geom_type == 'Polygon':
            geom = MultiPolygon([geom])
        return WKTElement(geom.wkt, srid)
    else:
        return None

srid = 4283

# Apply the function to each geometry
sa2['geom'] = sa2['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid))

# Drop the original geometry column
sa2 = sa2.drop(columns="geometry")



In [4]:
# filtering for only Greater Sydney Regions
sa2 = sa2[sa2.GCC_NAME21 == 'Greater Sydney']

In [5]:
import logging  # For logging information and errors

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Function to execute SQL commands with transaction handling
def execute_sql(connection, query):
    trans = connection.begin()  # Start a transaction
    try:
        connection.execute(text(query))
        trans.commit()  # Commit the transaction
        logging.info("SQL executed successfully: %s", query)
    except Exception as e:
        trans.rollback()  # Rollback the transaction if there's an error
        logging.error("Error executing SQL: %s", e)

# Create a connection to the database
credentials = "Credentials.json"
db, conn = pgconnect(credentials)  # Ensure this function correctly sets up the connection

# Creating the regions tables
execute_sql(conn, """
DROP TABLE IF EXISTS sa2_regions CASCADE;
CREATE TABLE sa2_regions (
    sa2_code CHAR(9) PRIMARY KEY,
    sa2_name VARCHAR(50),
    change_flag INTEGER,
    change_label VARCHAR(11),
    sa3_code CHAR(5),
    sa3_name VARCHAR(50),
    sa4_code CHAR(3),
    sa4_name VARCHAR(50),
    gcc_code CHAR(5),
    gcc_name VARCHAR(50),
    ste_code CHAR(1),
    ste_name VARCHAR(50),
    aus_code CHAR(3),
    aus_name VARCHAR(50),
    area_sq_km NUMERIC,
    loci_uri VARCHAR(1000),
    geometry GEOMETRY(MULTIPOLYGON, 4283)
);
""")


2024-05-14 21:47:58,155 - INFO - SQL executed successfully: 
DROP TABLE IF EXISTS sa2_regions CASCADE;
CREATE TABLE sa2_regions (
    sa2_code CHAR(9) PRIMARY KEY,
    sa2_name VARCHAR(50),
    change_flag INTEGER,
    change_label VARCHAR(11),
    sa3_code CHAR(5),
    sa3_name VARCHAR(50),
    sa4_code CHAR(3),
    sa4_name VARCHAR(50),
    gcc_code CHAR(5),
    gcc_name VARCHAR(50),
    ste_code CHAR(1),
    ste_name VARCHAR(50),
    aus_code CHAR(3),
    aus_name VARCHAR(50),
    area_sq_km NUMERIC,
    loci_uri VARCHAR(1000),
    geometry GEOMETRY(MULTIPOLYGON, 4283)
);



Connected successfully.


In [6]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [7]:
# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = sa2.select_dtypes(include=['number']).columns
sa2[numeric_cols] = sa2[numeric_cols].fillna(0)

# Selecting all columns that have data type 'object' which are generally strings, excluding geometry
string_cols = sa2.select_dtypes(include='object').columns.difference(['geom'])
sa2[string_cols] = sa2[string_cols].fillna('Unknown')

# Define a dictionary that maps old column names to new ones
column_mapping = {
    'SA2_CODE21': 'sa2_code',
    'SA2_NAME21': 'sa2_name',
    'CHG_FLAG21': 'change_flag',
    'CHG_LBL21': 'change_label',
    'SA3_CODE21': 'sa3_code',
    'SA3_NAME21': 'sa3_name',
    'SA4_CODE21': 'sa4_code',
    'SA4_NAME21': 'sa4_name',
    'GCC_CODE21': 'gcc_code',
    'GCC_NAME21': 'gcc_name',
    'STE_CODE21': 'ste_code',
    'STE_NAME21': 'ste_name',
    'AUS_CODE21': 'aus_code',
    'AUS_NAME21': 'aus_name',
    'AREASQKM21': 'area_sq_km',
    'LOCI_URI21': 'loci_uri',
    'geom': 'geometry'  # Ensure 'geom' is the correct geometry column name in the DataFrame
}

# Rename the columns in the DataFrame
sa2.rename(columns=column_mapping, inplace=True)

# Ensure the DataFrame columns are in the exact order as the SQL table
sa2 = sa2[['sa2_code', 'sa2_name', 'change_flag', 'change_label', 'sa3_code', 'sa3_name',
           'sa4_code', 'sa4_name', 'gcc_code', 'gcc_name', 'ste_code', 'ste_name',
           'aus_code', 'aus_name', 'area_sq_km', 'loci_uri', 'geometry']]


In [8]:
from sqlalchemy.dialects.postgresql import BYTEA  # If you're using PostgreSQL
from sqlalchemy import text
from geoalchemy2.shape import to_shape, from_shape



# Use the existing connection to insert data, making sure to use 'geometry' for the column name if required by the table schema
try:
    sa2.to_sql('sa2_regions', conn, if_exists='append', index=False,
               dtype={'geometry': Geometry('MULTIPOLYGON', srid=4283)})  # specifying dtype is crucial
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))


Data inserted successfully


In [9]:
query(conn, text("select * from sa2_regions"))

,sa2_code,sa2_name,change_flag,change_label,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,area_sq_km,loci_uri,geometry
0,102011028,Avoca Beach - Copacabana,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,6.4376,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000005E...
1,102011029,Box Head - MacMasters Beach,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,32.0802,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000010...
2,102011030,Calga - Kulnura,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,767.9512,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000200000001030000000100000085...
3,102011031,Erina - Green Point,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,33.7934,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000041...
4,102011032,Gosford - Springfield,0,No change,10201,Gosford,102,Central Coast,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,16.9123,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000007E...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,139.3336,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000046...
369,128021538,Sutherland - Kirrawee,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,7.7550,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB1000000100000001030000000100000089...
370,128021607,Engadine,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,8.9538,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB100000010000000103000000010000008E...
371,128021608,Loftus - Yarrawarrah,0,No change,12802,Sutherland - Menai - Heathcote,128,Sydney - Sutherland,1GSYD,Greater Sydney,1,New South Wales,AUS,Australia,3.8436,http://linked.data.gov.au/dataset/asgsed3/SA2/...,0106000020BB10000001000000010300000001000000A1...


## Create table for business

In [10]:
business = pd.read_csv('Businesses.csv')
# only dealing with businesses in the Greater Sydney region
business_GSA = business['sa2_code'].isin(pd.to_numeric(sa2['sa2_code']))
business = business[business_GSA]
business[business['sa2_code'] == 102011028]

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
28,A,"Agriculture, Forestry and Fishing",102011028,Avoca Beach - Copacabana,3,0,3,0,0,0,6
671,B,Mining,102011028,Avoca Beach - Copacabana,0,0,0,0,0,0,3
1314,C,Manufacturing,102011028,Avoca Beach - Copacabana,3,7,9,0,0,0,18
1957,D,"Electricity, Gas, Water and Waste Services",102011028,Avoca Beach - Copacabana,0,0,0,0,0,0,5
2600,E,Construction,102011028,Avoca Beach - Copacabana,25,83,78,3,0,0,187
3243,F,Wholesale Trade,102011028,Avoca Beach - Copacabana,4,6,11,0,0,3,24
3886,G,Retail Trade,102011028,Avoca Beach - Copacabana,14,12,15,4,0,0,45
4529,H,Accommodation and Food Services,102011028,Avoca Beach - Copacabana,7,4,17,3,0,0,31
5172,I,"Transport, Postal and Warehousing",102011028,Avoca Beach - Copacabana,7,3,6,0,0,0,15
5815,J,Information Media and Telecommunications,102011028,Avoca Beach - Copacabana,5,5,4,0,0,0,15


In [11]:
business_table_sql = """
DROP TABLE IF EXISTS businesses;
CREATE TABLE businesses (
    industry_code CHAR(1),
    industry_name VARCHAR(255),
    sa2_code CHAR(9) REFERENCES sa2_regions (sa2_code),
    sa2_name VARCHAR(50),
    "0_to_50k_businesses" INTEGER,
    "50k_to_200k_businesses" INTEGER,
    "200k_to_2m_businesses" INTEGER,
    "2m_to_5m_businesses" INTEGER,
    "5m_to_10m_businesses" INTEGER,
    "10m_or_more_businesses" INTEGER,
    total_businesses INTEGER,
    PRIMARY KEY (industry_code, sa2_code)
);
"""

# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = business.select_dtypes(include=['number']).columns
business[numeric_cols] = business[numeric_cols].fillna(0)

try:
    # Execute the SQL command to create the table
    conn.execute(text(business_table_sql))
    print("Table created successfully")
except Exception as e:
    print("An error occurred:", e)

Table created successfully


In [12]:
try:
    business.to_sql('businesses', conn, if_exists='append', index=False)
    print("Data inserted successfully")
    query(conn, text("select * from businesses"))
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))

Data inserted successfully


# Create table for Stops

In [13]:
stops = pd.read_csv('Stops.txt')
stops

,stop_id,stop_code,stop_name,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding,platform_code
0,200039,200039.0,"Central Station, Eddy Av, Stand A",-33.882206,151.206665,NaN,200060,0,NaN
1,200054,200054.0,"Central Station, Eddy Av, Stand D",-33.882042,151.206991,NaN,200060,0,NaN
2,200060,NaN,Central Station,-33.884084,151.206292,1.0,NaN,0,NaN
3,201510,NaN,Redfern Station,-33.891690,151.198866,1.0,NaN,0,NaN
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",-33.893329,151.198882,NaN,201510,0,NaN
...,...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",-33.822016,151.078797,NaN,21271,1,B
114714,2137185,2137185.0,"Cabarita Wharf, Side A",-33.840669,151.116926,NaN,21371,1,1A
114715,2137186,2137186.0,"Cabarita Wharf, Side B",-33.840769,151.116899,NaN,21371,1,1B
114716,21501,21501.0,Parramatta Wharf,-33.813904,151.010577,NaN,2150112,1,NaN


In [14]:
stops['geom'] = gpd.points_from_xy(stops.stop_lon, stops.stop_lat)  # creating the geometry column
stops = stops.drop(columns=['stop_lat', 'stop_lon'])  # removing the old latitude/longitude fields
stops

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",NaN,200060,0,NaN,POINT (151.20666 -33.88221)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",NaN,200060,0,NaN,POINT (151.20699 -33.88204)
2,200060,NaN,Central Station,1.0,NaN,0,NaN,POINT (151.20629 -33.88408)
3,201510,NaN,Redfern Station,1.0,NaN,0,NaN,POINT (151.19887 -33.89169)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",NaN,201510,0,NaN,POINT (151.19888 -33.89333)
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",NaN,21271,1,B,POINT (151.07880 -33.82202)
114714,2137185,2137185.0,"Cabarita Wharf, Side A",NaN,21371,1,1A,POINT (151.11693 -33.84067)
114715,2137186,2137186.0,"Cabarita Wharf, Side B",NaN,21371,1,1B,POINT (151.11690 -33.84077)
114716,21501,21501.0,Parramatta Wharf,NaN,2150112,1,NaN,POINT (151.01058 -33.81390)


In [15]:
from shapely.wkt import loads

# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = stops.select_dtypes(include=['number']).columns
stops[numeric_cols] = stops[numeric_cols].fillna(0)

# Selecting all columns that have data type 'object' which are generally strings, excluding geometry
string_cols = stops.select_dtypes(include='object').columns.difference(['geom'])
stops[string_cols] = stops[string_cols].fillna('Unknown')


srid = 4283
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))


stops

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geom
0,200039,200039.0,"Central Station, Eddy Av, Stand A",0.0,200060,0,Unknown,POINT (151.20666465471 -33.8822064874687)
1,200054,200054.0,"Central Station, Eddy Av, Stand D",0.0,200060,0,Unknown,POINT (151.20699145565 -33.8820421431408)
2,200060,0.0,Central Station,1.0,Unknown,0,Unknown,POINT (151.206292455081 -33.8840842535493)
3,201510,0.0,Redfern Station,1.0,Unknown,0,Unknown,POINT (151.198866071817 -33.8916900512711)
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",0.0,201510,0,Unknown,POINT (151.198881722942 -33.8933293130144)
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",0.0,21271,1,B,POINT (151.07879697831 -33.8220164586429)
114714,2137185,2137185.0,"Cabarita Wharf, Side A",0.0,21371,1,1A,POINT (151.116926480557 -33.8406690716775)
114715,2137186,2137186.0,"Cabarita Wharf, Side B",0.0,21371,1,1B,POINT (151.116898892402 -33.8407691073139)
114716,21501,21501.0,Parramatta Wharf,0.0,2150112,1,Unknown,POINT (151.010576673346 -33.8139042429414)


In [16]:
stops_table_sql = """
DROP TABLE IF EXISTS stops;
CREATE TABLE Stops (
    stop_id VARCHAR(255) PRIMARY KEY,
    stop_code VARCHAR(255),
    stop_name VARCHAR(255),
    location_type VARCHAR(255),
    parent_station VARCHAR(255),
    wheelchair_boarding INT,
    platform_code VARCHAR(255),
    geometry GEOMETRY(POINT,4283)
)
"""

try:
    # Execute the SQL command to create the table
    conn.execute(text(stops_table_sql))
    print("Table created successfully")
except Exception as e:
    print("An error occurred:", e)

Table created successfully


In [17]:
stops.rename(columns={
    'stop_id': 'stop_id',
    'stop_code': 'stop_code',
    'stop_name': 'stop_name',
    'location_type': 'location_type',
    'parent_station': 'parent_station',
    'wheelchair_boarding': 'wheelchair_boarding',
    'platform_code': 'platform_code',
    # If 'geom' needs special handling, make sure to process it before this step
    'geom': 'geometry'
}, inplace=True)

try:
    stops.to_sql('stops', conn, if_exists='append', index=False,
               dtype={'geometry': Geometry('POINT', srid)})  # specifying dtype is crucial
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))

Data inserted successfully


In [18]:
query(conn, text("select * from stops"))

,stop_id,stop_code,stop_name,location_type,parent_station,wheelchair_boarding,platform_code,geometry
0,200039,200039.0,"Central Station, Eddy Av, Stand A",0.0,200060,0,Unknown,0101000020BB100000FFA631FF9CE66240A1FF6524ECF0...
1,200054,200054.0,"Central Station, Eddy Av, Stand D",0.0,200060,0,Unknown,0101000020BB1000002F928BAC9FE66240E33DC7C1E6F0...
2,200060,0.0,Central Station,1.0,Unknown,0,Unknown,0101000020BB100000817FA2F299E662408FF33DAC29F1...
3,201510,0.0,Redfern Station,1.0,Unknown,0,Unknown,0101000020BB1000009E57611C5DE6624060304CE622F2...
4,201646,201646.0,"Redfern Station, Gibbons St, Stand B",0.0,201510,0,Unknown,0101000020BB100000DBF9333D5DE662403DFA6B9D58F2...
...,...,...,...,...,...,...,...,...
114713,212753,212753.0,"Sydney Olympic Park Wharf, Side B",0.0,21271,1,B,0101000020BB100000AF9B3D8185E262408F52D7D537E9...
114714,2137185,2137185.0,"Cabarita Wharf, Side A",0.0,21371,1,1A,0101000020BB100000EB409ADCBDE3624089CE4C0B9BEB...
114715,2137186,2137186.0,"Cabarita Wharf, Side B",0.0,21371,1,1B,0101000020BB100000C4F9BEA2BDE362403EB375529EEB...
114716,21501,21501.0,Parramatta Wharf,0.0,2150112,1,Unknown,0101000020BB100000E443E4A456E0624025C1A4032EE8...


# create table for Polls

In [19]:
polls = pd.read_csv('PollingPlaces2019.csv')

In [20]:
from shapely.wkt import loads

# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = polls.select_dtypes(include=['number']).columns
polls[numeric_cols] = polls[numeric_cols].fillna(0)

# Selecting all columns that have data type 'object' which are generally strings, excluding geometry
string_cols = polls.select_dtypes(include='object').columns.difference(['the_geom'])
polls[string_cols] = polls[string_cols].fillna('Unknown')

polls

,FID,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,latitude,longitude,the_geom
0,aec_federal_election_polling_places_2019.fid-4...,NSW,104,Barton,33595,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,0.000000,0.000000,NaN
1,aec_federal_election_polling_places_2019.fid-4...,NSW,105,Bennelong,33596,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,0.000000,0.000000,NaN
2,aec_federal_election_polling_places_2019.fid-4...,NSW,107,Blaxland,33600,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,0.000000,0.000000,NaN
3,aec_federal_election_polling_places_2019.fid-4...,NSW,109,Calare,33603,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,ORANGE,NSW,2800.0,0.000000,0.000000,NaN
4,aec_federal_election_polling_places_2019.fid-4...,NSW,113,Cowper,33716,2,Special Hospital Team 2,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,Unknown,Unknown,BARRACK HEIGHTS,NSW,2528.0,-34.564200,150.858000,POINT (-34.5642 150.858)
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,Unknown,Unknown,Unknown,WARILLA,NSW,2528.0,-34.550823,150.859755,POINT (-34.5508228 150.8597546)
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,Unknown,Unknown,WELBY,NSW,2575.0,-34.440900,150.424000,POINT (-34.4409 150.424)
2928,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,564,1,Windang,Windang Public School,60-64 Oakland Ave,Unknown,Unknown,WINDANG,NSW,2528.0,-34.531600,150.866000,POINT (-34.5316 150.866)


In [21]:
from shapely import wkt

# Function to safely convert to WKTElement only if the value is a valid WKT format
def safe_wkt_element(geom_str, srid=4283):
    if pd.notnull(geom_str) and geom_str.startswith('POINT'):
        try:
            # Load string as a shapely geometry object if it is not 'nan' and starts with 'POINT'
            geom = wkt.loads(geom_str)
            return WKTElement(geom.wkt, srid=srid)  
        except Exception as e:
            # In case of any error during conversion, return None or handle as needed
            print(f"Error converting geometry: {geom_str} | Error: {str(e)}")
            return None
    else:
        return None

# Ensure that the_geom column is of type string and not float
polls['the_geom'] = polls['the_geom'].astype(str)

# Apply the conversion to the 'the_geom' column
polls['the_geom'] = polls['the_geom'].apply(lambda x: safe_wkt_element(x))

# Now your dataframe should have the 'the_geom' column with proper geoalchemy2 objects or None
print(polls[['the_geom']])


                             the_geom
0                                None
1                                None
2                                None
3                                None
4                                None
...                               ...
2925         POINT (-34.5642 150.858)
2926  POINT (-34.5508228 150.8597546)
2927         POINT (-34.4409 150.424)
2928         POINT (-34.5316 150.866)
2929                             None

[2930 rows x 1 columns]


In [22]:
polls = polls.drop(columns=['latitude', 'longitude']) 
polls

,FID,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,the_geom
0,aec_federal_election_polling_places_2019.fid-4...,NSW,104,Barton,33595,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
1,aec_federal_election_polling_places_2019.fid-4...,NSW,105,Bennelong,33596,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
2,aec_federal_election_polling_places_2019.fid-4...,NSW,107,Blaxland,33600,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
3,aec_federal_election_polling_places_2019.fid-4...,NSW,109,Calare,33603,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,ORANGE,NSW,2800.0,None
4,aec_federal_election_polling_places_2019.fid-4...,NSW,113,Cowper,33716,2,Special Hospital Team 2,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,Unknown,Unknown,BARRACK HEIGHTS,NSW,2528.0,POINT (-34.5642 150.858)
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,Unknown,Unknown,Unknown,WARILLA,NSW,2528.0,POINT (-34.5508228 150.8597546)
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,Unknown,Unknown,WELBY,NSW,2575.0,POINT (-34.4409 150.424)
2928,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,564,1,Windang,Windang Public School,60-64 Oakland Ave,Unknown,Unknown,WINDANG,NSW,2528.0,POINT (-34.5316 150.866)


In [23]:
from shapely import wkt
from geoalchemy2.shape import to_shape

def safe_wkt_conversion(geom):
    if pd.notnull(geom) and isinstance(geom, str) and geom.startswith(('POINT', 'LINESTRING', 'POLYGON')):
        try:
            shape = wkt.loads(geom)
            return shape.wkt
        except Exception as e:
            print(f"Error in converting geometry: {str(e)}")
            return None
    else:
        return None

# Assuming 'the_geom' column exists and may contain WKT strings or NaNs
polls['the_geom'] = polls['the_geom'].apply(safe_wkt_conversion)

# Check and fill None for safe SQL insertion
#polls['the_geom'].fillna('POINT EMPTY', inplace=True)  # Use a placeholder for empty geometries



In [24]:
import logging  # For logging information and errors

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Function to execute SQL commands with transaction handling
def execute_sql(connection, query):
    trans = connection.begin()  # Start a transaction
    try:
        connection.execute(text(query))
        trans.commit()  # Commit the transaction
        logging.info("SQL executed successfully: %s", query)
    except Exception as e:
        trans.rollback()  # Rollback the transaction if there's an error
        logging.error("Error executing SQL: %s", e)

# Create a connection to the database
credentials = "Credentials.json"
db, conn = pgconnect(credentials)  # Ensure this function correctly sets up the connection

# Example SQL commands to create schema and manage tables
execute_sql(conn, """
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
        fid VARCHAR(255),
        state VARCHAR(255),
        division_id INT,
        division_name VARCHAR(255),
        polling_place_id INT PRIMARY KEY,
        polling_place_type_id INT,
        polling_place_name VARCHAR(255),
        premises_name VARCHAR(255),
        premises_address_1 VARCHAR(255),
        premises_address_2 VARCHAR(255),
        premises_address_3 VARCHAR(255),
        premises_suburb VARCHAR(255),
        premises_state_abbreviation VARCHAR(255),
        premises_post_code INT,
        geometry GEOMETRY(POINT, 4283)
);
""")
execute_sql(conn, "SET search_path TO sydney, public;")

2024-05-14 21:48:10,635 - INFO - SQL executed successfully: 
DROP TABLE IF EXISTS polls;
CREATE TABLE polls (
        fid VARCHAR(255),
        state VARCHAR(255),
        division_id INT,
        division_name VARCHAR(255),
        polling_place_id INT PRIMARY KEY,
        polling_place_type_id INT,
        polling_place_name VARCHAR(255),
        premises_name VARCHAR(255),
        premises_address_1 VARCHAR(255),
        premises_address_2 VARCHAR(255),
        premises_address_3 VARCHAR(255),
        premises_suburb VARCHAR(255),
        premises_state_abbreviation VARCHAR(255),
        premises_post_code INT,
        geometry GEOMETRY(POINT, 4283)
);

2024-05-14 21:48:10,637 - INFO - SQL executed successfully: SET search_path TO sydney, public;


Connected successfully.


In [25]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.4 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


In [26]:
polls.rename(columns={
    'FID': 'fid',  # Assuming 'FID' is the correct name in your DataFrame
    'state': 'state',
    'division_id': 'division_id',
    'division_name': 'division_name',
    'polling_place_id': 'polling_place_id',
    'polling_place_type_id': 'polling_place_type_id',
    'polling_place_name': 'polling_place_name',
    'premises_name': 'premises_name',
    'premises_address_1': 'premises_address_1',
    'premises_address_2': 'premises_address_2',
    'premises_address_3': 'premises_address_3',
    'premises_suburb': 'premises_suburb',
    'premises_state_abbreviation': 'premises_state_abbreviation',
    'premises_post_code': 'premises_post_code',
    'the_geom': 'geometry'  # This assumes the geometric column is named 'the_geom' in your DataFrame
}, inplace=True)
polls

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,geometry
0,aec_federal_election_polling_places_2019.fid-4...,NSW,104,Barton,33595,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
1,aec_federal_election_polling_places_2019.fid-4...,NSW,105,Bennelong,33596,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
2,aec_federal_election_polling_places_2019.fid-4...,NSW,107,Blaxland,33600,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
3,aec_federal_election_polling_places_2019.fid-4...,NSW,109,Calare,33603,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,ORANGE,NSW,2800.0,None
4,aec_federal_election_polling_places_2019.fid-4...,NSW,113,Cowper,33716,2,Special Hospital Team 2,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,Unknown,Unknown,BARRACK HEIGHTS,NSW,2528.0,None
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,Unknown,Unknown,Unknown,WARILLA,NSW,2528.0,None
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,Unknown,Unknown,WELBY,NSW,2575.0,None
2928,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,564,1,Windang,Windang Public School,60-64 Oakland Ave,Unknown,Unknown,WINDANG,NSW,2528.0,None


In [27]:
from sqlalchemy.dialects.postgresql import BYTEA  # If you're using PostgreSQL
from sqlalchemy import text
from geoalchemy2.shape import to_shape, from_shape



# Use the existing connection to insert data, making sure to use 'geometry' for the column name if required by your table schema
try:
    polls.to_sql('polls', conn, if_exists='append', index=False,
               dtype={'the_geom': Geometry('POINT', 4283)})  # specifying dtype is crucial
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))

Data inserted successfully


In [28]:
query(conn, "select * from polls")

,fid,state,division_id,division_name,polling_place_id,polling_place_type_id,polling_place_name,premises_name,premises_address_1,premises_address_2,premises_address_3,premises_suburb,premises_state_abbreviation,premises_post_code,geometry
0,aec_federal_election_polling_places_2019.fid-4...,NSW,104,Barton,33595,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0,None
1,aec_federal_election_polling_places_2019.fid-4...,NSW,105,Bennelong,33596,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0,None
2,aec_federal_election_polling_places_2019.fid-4...,NSW,107,Blaxland,33600,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0,None
3,aec_federal_election_polling_places_2019.fid-4...,NSW,109,Calare,33603,2,Special Hospital Team 1,Multiple sites,Unknown,Unknown,Unknown,ORANGE,NSW,2800,None
4,aec_federal_election_polling_places_2019.fid-4...,NSW,113,Cowper,33716,2,Special Hospital Team 2,Multiple sites,Unknown,Unknown,Unknown,Unknown,NSW,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,2809,1,Warilla South,Warilla High School,10 Keross Ave,Unknown,Unknown,BARRACK HEIGHTS,NSW,2528,None
2926,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,58798,5,Warilla WHITLAM PPVC,2/144 Shellharbour Rd,Unknown,Unknown,Unknown,WARILLA,NSW,2528,None
2927,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,31242,1,Welby,Welby Community Hall,14 Currockbilly St,Unknown,Unknown,WELBY,NSW,2575,None
2928,aec_federal_election_polling_places_2019.fid-4...,NSW,150,Whitlam,564,1,Windang,Windang Public School,60-64 Oakland Ave,Unknown,Unknown,WINDANG,NSW,2528,None


# create table for schools

In [29]:
school_primary=gpd.read_file("Catchments/catchments_primary.shp")
school_primary

# Replacing all 'Y'  with 1 and 'N' with 0
year_column_list = ['KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12']
for year in year_column_list:
    school_primary[year] = school_primary[year].map({'Y': 1, 'N': 0})
    
# converting all of these columns to numeric

for year in year_column_list:
    school_primary[year] = pd.to_numeric(school_primary[year])

In [30]:
school_secondary=gpd.read_file("Catchments/catchments_secondary.shp")
school_secondary


# Replacing all 'Y'  with 1 and 'N' with 0
year_column_list = ['KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12']
for year in year_column_list:
    school_secondary[year] = school_secondary[year].map({'Y': 1, 'N': 0})
    
# converting all of these columns to numeric

for year in year_column_list:
    school_secondary[year] = pd.to_numeric(school_secondary[year])

In [31]:
schools = pd.concat([school_primary, school_secondary], ignore_index=True)
#dropping duplicates
schools.drop_duplicates(inplace=True)


In [32]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

srid = 4283

schoolsCopy = schools.copy()  # creating a copy of the original for later
schools['geom'] = schools['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
schools = schools.drop(columns="geometry")  # deleting the old copy
schools.rename(columns = {'geom':'geometry'}, inplace = True)
schools

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,None,MULTIPOLYGON (((150.93563850416004 -33.7161211...
1,2404,PRIMARY,Lindfield EPS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,None,MULTIPOLYGON (((151.1833640465581 -33.74748398...
2,4393,PRIMARY,Carlingford WPS,20220223,1,1,1,1,1,1,1,0,0,0,0,0,0,None,MULTIPOLYGON (((151.0451821055135 -33.77303212...
3,4615,PRIMARY,Caddies Ck PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,None,MULTIPOLYGON (((150.92567327976582 -33.7296030...
4,3918,PRIMARY,Killara PS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,None,MULTIPOLYGON (((151.1537883781186 -33.75586174...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,8213,HIGH_BOYS,Birrong BHS,20211221,0,0,0,0,0,0,0,1,1,1,1,1,1,None,MULTIPOLYGON (((151.05363508494244 -33.8507612...
2094,8108,HIGH_COED,Cessnock HS,20230405,0,0,0,0,0,0,0,1,1,1,1,1,1,None,MULTIPOLYGON (((151.4285214252146 -32.74415247...
2095,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,None,MULTIPOLYGON (((143.37723402388323 -34.8017260...
2096,1115,CENTRAL_HIGH,Balranald CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,None,MULTIPOLYGON (((143.65540852703225 -33.5570248...


In [33]:
db, conn = pgconnect(credentials)

schools_table_sql = """
DROP TABLE IF EXISTS schools;
CREATE TABLE schools (
    USE_ID CHAR(4),
    CATCH_TYPE VARCHAR(255),
    USE_DESC VARCHAR(255),
    ADD_DATE VARCHAR(255),
    KINDERGART INT,
    YEAR1 INT,
    YEAR2 INT,
    YEAR3 INT,
    YEAR4 INT,
    YEAR5 INT,
    YEAR6 INT,
    YEAR7 INT,
    YEAR8 INT,
    YEAR9 INT,
    YEAR10 INT,
    YEAR11 INT,
    YEAR12 INT,
    PRIORITY VARCHAR(255),
    geometry GEOMETRY(MULTIPOLYGON, 4283),
    PRIMARY KEY (USE_ID, CATCH_TYPE)
);
"""

try:
    # Execute the SQL command to create the table
    conn.execute(text(schools_table_sql))
    print("Table created successfully")
except Exception as e:
    print("An error occurred:", e)

Connected successfully.
Table created successfully


In [34]:
import numpy as np

# Replace 'None' strings and None objects with np.nan
schools['PRIORITY'] = schools['PRIORITY'].replace(['None', None], np.nan)
schools['PRIORITY'].fillna('Unknown', inplace=True)

# Display unique values in 'PRIORITY' to confirm changes
print(schools['PRIORITY'].unique())

['Unknown' 'N' 'Y']


In [35]:
# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = schools.select_dtypes(include=['number']).columns
schools[numeric_cols] = schools[numeric_cols].fillna(0)

# Selecting all columns that have data type 'object' which are generally strings, excluding geometry
string_cols = schools.select_dtypes(include='object').columns.difference(['geom'])
schools[string_cols] = schools[string_cols].fillna('Unknown')



In [36]:
schools

,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,MULTIPOLYGON (((150.93563850416004 -33.7161211...
1,2404,PRIMARY,Lindfield EPS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,MULTIPOLYGON (((151.1833640465581 -33.74748398...
2,4393,PRIMARY,Carlingford WPS,20220223,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,MULTIPOLYGON (((151.0451821055135 -33.77303212...
3,4615,PRIMARY,Caddies Ck PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,MULTIPOLYGON (((150.92567327976582 -33.7296030...
4,3918,PRIMARY,Killara PS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,MULTIPOLYGON (((151.1537883781186 -33.75586174...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,8213,HIGH_BOYS,Birrong BHS,20211221,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,MULTIPOLYGON (((151.05363508494244 -33.8507612...
2094,8108,HIGH_COED,Cessnock HS,20230405,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,MULTIPOLYGON (((151.4285214252146 -32.74415247...
2095,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,MULTIPOLYGON (((143.37723402388323 -34.8017260...
2096,1115,CENTRAL_HIGH,Balranald CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,MULTIPOLYGON (((143.65540852703225 -33.5570248...


In [37]:
# Renaming DataFrame columns to match the SQL table column names exactly
schools.rename(columns={
    'USE_ID': 'use_id',
    'CATCH_TYPE': 'catch_type',
    'USE_DESC': 'use_desc',
    'ADD_DATE': 'add_date',
    'KINDERGART': 'kindergart',
    'YEAR1': 'year1',
    'YEAR2': 'year2',
    'YEAR3': 'year3',
    'YEAR4': 'year4',
    'YEAR5': 'year5',
    'YEAR6': 'year6',
    'YEAR7': 'year7',
    'YEAR8': 'year8',
    'YEAR9': 'year9',
    'YEAR10': 'year10',
    'YEAR11': 'year11',
    'YEAR12': 'year12',
    'PRIORITY': 'priority',
    'geometry': 'geometry'  # Make sure the geometry column name matches too
}, inplace=True)

# Attempt to insert data again
try:
    schools.to_sql('schools', conn, if_exists='append', index=False, dtype={'geometry': Geometry('MULTIPOLYGON', srid=4283)})
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))

Data inserted successfully


In [38]:
query(conn, text("select * from schools"))

,use_id,catch_type,use_desc,add_date,kindergart,year1,year2,year3,year4,year5,year6,year7,year8,year9,year10,year11,year12,priority,geometry
0,2838,PRIMARY,Parklea PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,0106000020BB1000000100000001030000000100000078...
1,2404,PRIMARY,Lindfield EPS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,0106000020BB10000001000000010300000001000000BE...
2,4393,PRIMARY,Carlingford WPS,20220223,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,0106000020BB1000000100000001030000000100000065...
3,3908,PRIMARY,Lane Cove WPS,20211219,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,0106000020BB1000000100000001030000000100000034...
4,4615,PRIMARY,Caddies Ck PS,20181210,1,1,1,1,1,1,1,0,0,0,0,0,0,Unknown,0106000020BB1000000100000001030000000100000056...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2093,8213,HIGH_BOYS,Birrong BHS,20211221,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,0106000020BB1000000100000001030000000100000040...
2094,8108,HIGH_COED,Cessnock HS,20230405,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,0106000020BB10000001000000010300000001000000AD...
2095,3235,CENTRAL_HIGH,Tooleybuc CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,0106000020BB100000010000000103000000010000003E...
2096,1115,CENTRAL_HIGH,Balranald CS,20200512,0,0,0,0,0,0,0,1,1,1,1,1,1,Unknown,0106000020BB100000010000000103000000010000005B...


# Create table for Population

In [39]:
population=pd.read_csv('Population.csv')
population_GSA = population['sa2_code'].isin(pd.to_numeric(sa2['sa2_code']))
population = population[population_GSA]
population

,sa2_code,sa2_name,0-4_people,5-9_people,10-14_people,15-19_people,20-24_people,25-29_people,30-34_people,35-39_people,...,45-49_people,50-54_people,55-59_people,60-64_people,65-69_people,70-74_people,75-79_people,80-84_people,85-and-over_people,total_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,572,602,570,520,464,369,226,142,70,7530
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,749,794,895,863,925,603,331,264,11052
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,325,436,422,397,327,264,190,100,75,4748
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,859,882,901,930,917,1065,976,773,1028,14803
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1330,1241,1377,1285,1166,949,664,476,537,21346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2,4,10,4,9,7,1,2,...,4,0,0,0,1,0,0,0,0,45
369,128021538,Sutherland - Kirrawee,1519,1292,1150,1117,1335,1852,2120,1945,...,1569,1391,1285,1157,909,909,781,582,807,23369
370,128021607,Engadine,1157,1283,1469,1209,891,675,928,1229,...,1315,1086,909,764,707,886,748,389,327,17379
371,128021608,Loftus - Yarrawarrah,503,487,575,508,380,293,426,493,...,564,477,450,387,418,335,263,192,109,7354


In [40]:
# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = population.select_dtypes(include=['number']).columns
population[numeric_cols] = population[numeric_cols].fillna(0)

# Selecting all columns that have data type 'object' which are generally strings
string_cols = population.select_dtypes(include='object').columns
population[string_cols] = population[string_cols].fillna('Unknown')

In [41]:
db, conn = pgconnect(credentials)

population_table_sql = """
DROP TABLE IF EXISTS population;
CREATE TABLE population (
    sa2_code CHAR(9) PRIMARY KEY REFERENCES sa2_regions (sa2_code),
    sa2_name VARCHAR(50),
    people_0_4 INT,
    people_5_9 INT,
    people_10_14 INT,
    people_15_19 INT,
    people_20_24 INT,
    people_25_29 INT,
    people_30_34 INT,
    people_35_39 INT,
    people_40_44 INT,
    people_45_49 INT,
    people_50_54 INT,
    people_55_59 INT,
    people_60_64 INT,
    people_65_69 INT,
    people_70_74 INT,
    people_75_79 INT,
    people_80_84 INT,
    people_85_over INT,
    total_people INT
);
"""

try:
    # Execute the SQL command to create the table
    conn.execute(text(population_table_sql))
    print("Table created successfully")
except Exception as e:
    print("An error occurred:", e)

Connected successfully.
Table created successfully


In [42]:
# Renaming DataFrame columns to match the SQL table column names exactly
population.rename(columns={
    'sa2_code': 'sa2_code',
    'sa2_name': 'sa2_name',
    '0-4_people': 'people_0_4',
    '5-9_people': 'people_5_9',
    '10-14_people': 'people_10_14',
    '15-19_people': 'people_15_19',
    '20-24_people': 'people_20_24',
    '25-29_people': 'people_25_29',
    '30-34_people': 'people_30_34',
    '35-39_people': 'people_35_39',
    '40-44_people': 'people_40_44',
    '45-49_people': 'people_45_49',
    '50-54_people': 'people_50_54',
    '55-59_people': 'people_55_59',
    '60-64_people': 'people_60_64',
    '65-69_people': 'people_65_69',
    '70-74_people': 'people_70_74',
    '75-79_people': 'people_75_79',
    '80-84_people': 'people_80_84',
    '85-and-over_people': 'people_85_over',
    'total_people': 'total_people'
}, inplace=True)

# Attempt to insert data again
try:
    population.to_sql('population', conn, if_exists='append', index=False)
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred: ", e)
    conn.execute(text('ROLLBACK'))

Data inserted successfully


In [43]:
query(conn, text("select * from population"))

,sa2_code,sa2_name,people_0_4,people_5_9,people_10_14,people_15_19,people_20_24,people_25_29,people_30_34,people_35_39,...,people_45_49,people_50_54,people_55_59,people_60_64,people_65_69,people_70_74,people_75_79,people_80_84,people_85_over,total_people
0,102011028,Avoca Beach - Copacabana,424,522,623,552,386,222,306,416,...,572,602,570,520,464,369,226,142,70,7530
1,102011029,Box Head - MacMasters Beach,511,666,702,592,461,347,420,535,...,749,749,794,895,863,925,603,331,264,11052
2,102011030,Calga - Kulnura,200,225,258,278,274,227,214,286,...,325,436,422,397,327,264,190,100,75,4748
3,102011031,Erina - Green Point,683,804,880,838,661,502,587,757,...,859,882,901,930,917,1065,976,773,1028,14803
4,102011032,Gosford - Springfield,1164,1044,1084,1072,1499,1864,1750,1520,...,1330,1241,1377,1285,1166,949,664,476,537,21346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,128021537,Royal National Park,2,4,10,4,9,7,1,2,...,4,0,0,0,1,0,0,0,0,45
369,128021538,Sutherland - Kirrawee,1519,1292,1150,1117,1335,1852,2120,1945,...,1569,1391,1285,1157,909,909,781,582,807,23369
370,128021607,Engadine,1157,1283,1469,1209,891,675,928,1229,...,1315,1086,909,764,707,886,748,389,327,17379
371,128021608,Loftus - Yarrawarrah,503,487,575,508,380,293,426,493,...,564,477,450,387,418,335,263,192,109,7354


# Create table for Income

In [44]:
income=pd.read_csv('Income.csv')

# only considering those regions in Greater Sydney
income_GSA = income['sa2_code21'].isin(pd.to_numeric(sa2['sa2_code']))
income = income[income_GSA]

income

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
28,102011028,Avoca Beach - Copacabana,4749,47,55065,77615
29,102011029,Box Head - MacMasters Beach,6636,49,51927,71509
30,102011030,Calga - Kulnura,2965,49,49168,63802
31,102011031,Erina - Green Point,8010,48,51905,71992
32,102011032,Gosford - Springfield,12051,41,54372,65283
...,...,...,...,...,...,...
637,128021537,Royal National Park,14,37,36980,47584
638,128021538,Sutherland - Kirrawee,13895,41,64940,74867
639,128021607,Engadine,10239,43,63695,72995
640,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


In [45]:
# Replace NaN in all numeric columns with 0, excluding geometry column
numeric_cols = income.select_dtypes(include=['number']).columns
income[numeric_cols] = income[numeric_cols].fillna(0)

# Replacing all 'np' in numeric columns with 0, i.e. all columns excluding sa2_name
numeric_cols = income.select_dtypes(include='object').columns.difference(['sa2_name'])
income[numeric_cols] = income[numeric_cols].replace('np', '0')

# Selecting all columns that have data type 'object' which are generally strings and filling NA with 'Unknown'
string_cols = income.select_dtypes(include='object').columns
income[string_cols] = income[string_cols].fillna('Unknown')

In [46]:
db, conn = pgconnect(credentials)

income_table_sql = """
DROP TABLE IF EXISTS income;
CREATE TABLE income (
    sa2_code CHAR(9) PRIMARY KEY REFERENCES sa2_regions (sa2_code),
    sa2_name VARCHAR(50),
    earners INT,
    median_age INT,
    median_income INT,
    mean_income INT
);
"""

try:
    # Execute the SQL command to create the table
    conn.execute(text(income_table_sql))
    print("Table created successfully")
except Exception as e:
    print("An error occurred:", e)

Connected successfully.
Table created successfully


In [47]:
# Renaming DataFrame columns to match the SQL table column names exactly
income.rename(columns={
    'sa2_code21': 'sa2_code',
    'sa2_name': 'sa2_name',
    'earners': 'earners',
    'median_age': 'median_age',
    'median_income': 'median_income',
    'mean_income': 'mean_income'
}, inplace=True)


try:
    income.to_sql('income', conn, if_exists='append', index=False)
    print("Data inserted successfully")
except Exception as e:
    print("An error occurred:", e)
    conn.execute('ROLLBACK')  # This might need to be adjusted based on your DBMS if it's not PostgreSQL


Data inserted successfully


In [48]:
query(conn, text("select * from income"))

,sa2_code,sa2_name,earners,median_age,median_income,mean_income
0,102011028,Avoca Beach - Copacabana,4749,47,55065,77615
1,102011029,Box Head - MacMasters Beach,6636,49,51927,71509
2,102011030,Calga - Kulnura,2965,49,49168,63802
3,102011031,Erina - Green Point,8010,48,51905,71992
4,102011032,Gosford - Springfield,12051,41,54372,65283
...,...,...,...,...,...,...
368,128021537,Royal National Park,14,37,36980,47584
369,128021538,Sutherland - Kirrawee,13895,41,64940,74867
370,128021607,Engadine,10239,43,63695,72995
371,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


Task 2